In [6]:
def createTestSet(loc_idx_1, loc_idx_2, coordinates):
    audio_index_1 = 15
    audio_index_2 = 5
    src_1 = AudioSignal(path=src_1_path[audio_index_1], slice_duration=frameDuration)
    src_2 = AudioSignal(path=src_2_path[audio_index_2], slice_duration=frameDuration)
    # binaural_sig = BinauralSignal(hrir=hrirSet, fs_hrir=fs_HRIR, fs_audio=src_1.fs_audio)
    # binaural_cues = BinauralCues(fs_audio=src_1.fs_audio, prep_method="standardise")

    slice_idx_1 = 0
    slice_idx_2 = 0
    count = 0
    while True:
        # print(f"Current audio (src 1): {audio_index_1}, and (src 2): {audio_index_2}")
        # print(f"Number of slices (audio 1): {len(src_1.slice_list)}, and (audio 2): {len(src_2.slice_list)}")
        if slice_idx_1 >= len(src_1.slice_list):
            slice_idx_1 = 0
            audio_index_1 += 1
            src_1 = AudioSignal(path=src_1_path[audio_index_1], slice_duration=frameDuration)
            
        if slice_idx_2 >= len(src_2.slice_list):
            slice_idx_2 = 0
            audio_index_2 += 1
            src_2 = AudioSignal(path=src_2_path[audio_index_2], slice_duration=frameDuration)
        

        sig_sliced_1 = src_1(idx=slice_idx_1)
        sig_sliced_1 = src_1.apply_gain(sig_sliced_1, target_power=-20)
        sig_sliced_2 = src_2(idx=slice_idx_2)
        sig_sliced_2 = src_2.apply_gain(sig_sliced_2, target_power=-20)

        sigL_1, sigR_1 = binaural_sig(sig_sliced_1, loc_idx_1)
        sigL_2, sigR_2 = binaural_sig(sig_sliced_2, loc_idx_2)
        magL, phaseL, magR, phaseR = binaural_cues(sigL_1+sigL_2, sigR_1+sigR_2)

        # save_cues(cuesList=[magL, phaseL, magR, phaseR], locIndex=[loc_idx_1, loc_idx_2])
        # if save_cues.fileCount == Nsample:
        #     return
        test_cues[count] = torch.tensor([magL, phaseL, magR, phaseR]).permute(1,2,0)

        if coordinates.lower() == "spherical":
            test_label[count] = torch.from_numpy(degree2Radian(np.concatenate((locLabel[loc_idx_1], locLabel[loc_idx_2]), axis=-1)))
        elif coordinates.lower() == "cartesian":
            test_label[count] = torch.from_numpy(
                np.concatenate(
                    (spherical2Cartesian(locLabel[loc_idx_1]), spherical2Cartesian(locLabel[loc_idx_2])),
                    axis=-1
                )
        )
        
        count += 1
        if count >= Nsample:
            return

        slice_idx_1 += 1
        slice_idx_2 += 1
        # print(count)

# Two-source confusion heatmap

In [13]:
from glob import glob
import os
import soundfile as sf
import torch
import torch.nn as nn

from load_data import *
from utils import *
from utils_model import *
from models import *
from loss import *

"""definition"""
Nfreq = 512
Ntime = 72
Ncues = 4
Nsound = 2
task = "allRegression"
whichDec = "src"
numEnc = 3
coordinates = "spherical"
frameDuration = 1
isHPC = True
num_workers = 0
Nsample = 1
batch_size = 32

"""directories"""
hrir_dir = "./HRTF"
model_dir = "D:/SSSL-D/model/HPC/1008_2Sound_src/"
src_1_path = glob(os.path.join("./audio_valid/speech_male" + "/*"))
src_2_path = glob(os.path.join("./audio_valid/speech_female" + "/*"))

"""load HRIR"""
path = hrir_dir + "/IRC*"
hrirSet, locLabel, fs_HRIR = loadHRIR(path)
"""create a tensor that stores all testing examples"""
test_cues = torch.empty((Nsample, Nfreq, Ntime, Ncues))
if coordinates.lower() == "spherical":
    test_label = torch.empty((Nsample, 2*Nsound))
elif coordinates.lower() == "cartesian":
    test_label = torch.empty((Nsample, 3*Nsound))
"""load model"""
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(
    task=task,
    Ntime=Ntime,
    Nfreq=Nfreq,
    Ncues=Ncues,
    Nsound=Nsound,
    whichEnc="diy",
    whichDec=whichDec,
    device=device,
    numEnc=numEnc,
    coordinates=coordinates
    # numFC=numFC,
)
if isHPC:
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model = model.to(device)
checkpoint = torch.load(model_dir+"param_bestValLoss.pth.tar")
model.load_state_dict(checkpoint['model'], strict=True)

cost_func = CostFunc(task=task, Nsound=Nsound, device=device)

src_1 = AudioSignal(path=src_1_path[0], slice_duration=frameDuration)
binaural_sig = BinauralSignal(hrir=hrirSet, fs_hrir=fs_HRIR, fs_audio=src_1.fs_audio)
binaural_cues = BinauralCues(fs_audio=src_1.fs_audio, prep_method="standardise")
loc_region = LocRegion(locLabel=locLabel)
loc_1 = [i for i in loc_region.elev_dict[0] if i in loc_region.low_left+loc_region.high_left]
loc_1 = loc_region.low_left+loc_region.high_left
loc_2 = loc_region.low_right+loc_region.high_right+loc_region.azim_dict[180]+loc_region.azim_dict[0]


confusion_heatmap = torch.empty(187,187)
for loc_idx_1 in range(0, len(loc_1), 1):
    for loc_idx_2 in range(0, len(loc_2), 1):
        print(f"plotting: {loc_idx_1} and {loc_idx_2}")
        createTestSet(loc_1[loc_idx_1], loc_2[loc_idx_2], coordinates=coordinates)

        dataset = TensorDataset(test_cues, test_label)

        test_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

        """model inference"""
        test_correct = 0.0
        test_total = 0.0
        test_sum_loss = 0.0
        test_loss = 0.0
        test_acc = 0.0
        model.eval()
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(test_loader, 0):
                inputs, labels = Variable(inputs).to(device), Variable(labels).to(device)
                outputs = model(inputs)
                if args.coordinates.lower() == "cartesian":
                    outputs = cartesian2Spherical(outputs)
                    labels = cartesian2Spherical(labels)
                test_loss = cost_func(outputs, labels)
                test_sum_loss += test_loss.item()
            test_loss = test_sum_loss / (i + 1)
            test_acc = radian2Degree(test_loss)
            print('Location : Test Loss: %.04f | RMS angle error (degree): %.04f '
                % (test_loss, test_acc))

[1002]
locLabel shape:  (187, 2)  (order: elev, azim)


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 6.00 GiB total capacity; 4.90 GiB already allocated; 0 bytes free; 4.95 GiB reserved in total by PyTorch)